In [1]:
##This is the diver for the generic bootstrap particle filter

from pylab import *
from scipy.integrate import odeint
from L63_V import L63
import ipdb

In [2]:
# %load particle_filter.py
def  bootstrap(model,state_dim,prior,ens_size,weights,interval,nanl,tanl,obs,Q):

    """This is a general bootstrap particle filter function
    
    The fields to supply are the derivative function `model', the initial cloud
    'initial', the model dimension and cloud size,
    the particle weights at initialization `weights', the interval 
    to integrate the cloud '`interval', the observation with which to 
    update the cloud at the end analysis time, and the inverse of the 
    observational error covaraince `Q'.
    
    We assume that the model is vectorized to accept a cloud of initial
    conditions of the shape [model_dimension, cloud_size]."""
    
    # store the analysis times indices in the full integration interval
    a_time = array(range(0,len(interval),tanl))

    # storage dictionary for the trajectories and weights
    p_series = {}
    A = 'A_'
    
    # divergence safety check
    divergence = False
    
    # loop through the analysis times starting at time zero
    for i in range(nanl):
        
        # recompute the weights and load into the dict as a tuple
        [analysis,weights,ens_size] = no_resample_update(weights,obs[i,:],Q,prior,ens_size,state_dim)
        
        # check for filter divergence
        if ens_size < 10:
            divergence = True
            A_i = A + str(i)
            p_series[A_i] = [prior,analysis,weights]
            break
        
        # integrate the initial cloud to the next analysis time;
        # note integration interval starts at time 0, and slice notation goes to the last index - 1
        traj = odeint(model,analysis,interval[a_time[i]:a_time[i+1]+1])
        
        #create storage for next iteration
        A_i = A + str(i)
        p_series[A_i] = [prior, analysis, weights, traj]
        
        #initialize the next forecast
        prior = traj[-1,:]
        
    # final analysis time weight update
    ipdb.set_trace()
    if not divergence:
        [analysis,weights,ens_size] = no_resample_update(weights,obs[i+1,:],Q,prior,ens_size,state_dim)
        A_i = A + str(i+1)
        p_series[A_i] = [prior, analysis, weights]
    
    return p_series

In [3]:
def gen_obs(model,truth,H,exp_int,nanl,tanl,obs_var):
    
    """This function generates a sequence of observations for analysis
    
    Given the model, the initialization of the truth, observational operator H,
    the interval of the experiment integration, the number of analyses, the 
    time interval between analyses, and the vector of observational variances (assuming
    covariances are zero), this function returns a sequence of observations given by perturbations
    of the the true trajectory."""
    
    # Define the model and obs dimensions
    [obs_dim,state_dim] = shape(H)
    
    # Propagate a `true' trajectory to generate the observations
    truth = odeint(L63,truth,exp_int)
    truth = reshape(truth,[exp_len+1,3])

    # Create observations with error, with first observation at time zero, and ranging to
    # time exp_len, at intervals of tanl
    obs = H.dot(truth[::tanl,:]) + (randn(nanl+1,state_dim)*sqrt(obs_var))
    return obs

In [4]:
## System Parameters

# Define the number of particles in the ensemble
root = 8
particle_number = root**3

# Spin time
spin_end = 1000

# Time step
dt = .01

# Spin interval
spin = linspace(0,spin_end,spin_end/dt)

# Obs Err variance (% of climate variance) 
obs_var = 0.01

# Analysis performed after tanl steps
tanl = 10

# Number of Analyses (after the analysis at time zero)
nanl = 1000

# Experiment length defined
exp_len = tanl*nanl

# Interval of integration including time zero
exp_int = linspace(0,exp_len*dt,exp_len+1)

# state dimension
state_dim = 3

In [ ]:
def attractor_spin(particle_number,root,model,state_dim,obs_var):
    
    """Spin up the ensemble of initial conditions for the filter and `truth'
    
    Assign cubes of initial conditions for the ensemble spin up, run the spin,calculate 
    spin mean, and variance to set initial conditions for the experiment.  The number
    of particles must be a cubic, and the cube root of the number of particles is supplied
    as `root'.""" 

    # Generate basepoint for the cube of initial conditions
    ens_base = randint(-100,100)

    # Define cube of initial conditions for the spin up 
    cube = zeros([particle_number,state_dim])

    for i in range(root):
        for j in range(root):
            for k in range(root):            
                cube[(root**2)*i + root*j + k, :] =(array([k*exp(-1),j*exp(-1),i*exp(-1)])+ens_base)
                                                      

    # Spin up the initial cloud
    spin_cloud = reshape(cube,3*particle_number)
    spin_cloud = odeint(L63,spin_cloud,spin)
    spin_cloud = reshape(spin_cloud,[len(spin),particle_number,state_dim])
    spun_cloud = spin_cloud[-1,:,:]

    ## Calcualte the environmental statistics

    #Determine the mean for the spin cloud at each time step
    spin_mean = mean(spun_cloud,axis=0)

    #Calculate variance of the mean along spin up
    cl_var = 2*var(spun_cloud,axis=0)
    obs_var = obs_var*cl_var

    #Observational Error covariance stored    
    R = eye(state_dim)*obs_var
    Q = inv(R)

    ## Create the truth for `perfect' model experiment

    # Generate random ensemble member to be initializaiton for `truth'
    P = randint(0,particle_number)
    truth = spun_cloud[P,:]

    # `Truth' state is eliminated from the particle cloud
    PF_cloud = delete(spun_cloud,P,0)
    
    return [truth,PF_cloud,Q]

In [ ]:
## Run bootstrap filter on the sequence of observations with the intialized cloud

# Set the weights and initial conditions
ens_size = len(PF_cloud[:,0])
weights = 1.0/(ens_size)
weights = weights*ones(ens_size)
%debug
PF_cloud = reshape(PF_cloud,PF_cloud.size)

pdf_series = bootstrap(L63,state_dim,PF_cloud,ens_size,weights,exp_int,nanl,tanl,obs,Q)


In [ ]:
attractor_spin(particle_number,root,L63,state_dim,obs_var)